## Uniform Time Course process output comparisons

In [1]:
import sys
import os
from tempfile import mkdtemp

import numpy as np
from process_bigraph import Composite
from process_bigraph.experiments.parameter_scan import RunProcess


sys.path.insert(0, '..')

In [2]:
from biosimulator_processes.io import read_report_outputs
from biosimulator_processes.helpers import plot_utc_outputs, plot_ode_output_data

In [3]:
from biosimulator_processes.processes.amici_process import UtcAmici
from biosimulator_processes.processes.copasi_process import UtcCopasi

In [4]:
omex_dir = '../test_suite/examples/sbml-core/Elowitz-Nature-2000-Repressilator'
omex_fp = omex_dir + '.omex'
sbml_fp = os.path.join(omex_dir, 'BIOMD0000000012_url.xml')
sedml_fp = os.path.join(omex_dir, 'simulation.sedml')
reports_path = os.path.join(omex_dir, 'reports.h5')

Configure and create an Amici UTC instance using just the archive filepath.

In [5]:
utc_amici = UtcAmici(config={'model': {'model_source': omex_dir + '.omex'}})

In [6]:
# t range parameters set via sedml introspection within archive contents:
utc_amici.t[0], utc_amici.t[-1], utc_amici.duration, utc_amici.step_size, utc_amici.num_steps

In [7]:
# you can verify this time config against the SEDML file
!cat ./examples/sbml-core/Elowitz-Nature-2000-Repressilator/simulation.sedml

In [8]:
# run the amici utc simulation with the interface's update method:
amici_results = utc_amici.update()

In [9]:
# use built-in plot to plot results
utc_amici.plot_results()

Create a `UtcTellurium` instance as above

In [10]:
from biosimulator_processes.processes.tellurium_process import UtcTellurium


utc_tellurium = UtcTellurium(config={'model': {'model_source': omex_dir + '.omex'}})

In [11]:
tellurium_results = utc_tellurium.update()

In [12]:
utc_tellurium._results['floating_species'].keys()

In [13]:
utc_tellurium.plot_results()

Create a `UtcCopasi` instance and run the same

In [14]:
utc_copasi = UtcCopasi(config={'model': {'model_source': omex_dir + '.omex'}})

In [15]:
copasi_results = utc_copasi.update()

In [16]:
copasi_results['time'][-1]

In [17]:
utc_copasi.plot_results()

In [18]:
utc_copasi._tc

Finally, optionally generate a "ground truth" which we include in the comparison. The default source for ground-truth data is via uploading an omex archive as model source. The report outputs must be included in the archive.

In [19]:
# Generate comparison matrix from ground truth
from biosimulator_processes.api.compare import write_utc_comparison_reports
from biosimulator_processes.io import standardize_report_outputs

In [20]:
report_results = standardize_report_outputs(reports_path)

In [21]:
print(f"copasi result keys: {copasi_results['floating_species'].keys()}\nTellurium result keys: {tellurium_results['floating_species'].keys()}\nAmici Result keys: {amici_results['floating_species'].keys()}\nGround truth report keys: {report_results['floating_species'].keys()}")

In [22]:
plot_ode_output_data(report_results, simulator_name='Ground Truth')

Generate a series of comparisons for each species and save to a given dir

In [23]:
utc_copasi.t[-1]

In [24]:
utc_copasi.step_size, utc_amici.step_size, utc_tellurium.step_size

In [25]:
save_dir = '../test_outputs/repressilator-utc-comparison'
if os.path.exists(save_dir):
    import shutil 
    shutil.rmtree(save_dir)
    
    
write_utc_comparison_reports(
    save_dir=save_dir,
    copasi_results=copasi_results,
    tellurium_results=tellurium_results,
    amici_results=amici_results,
    ground_truth_results=report_results,
    sbml_species_mapping=utc_copasi.sbml_species_mapping,
    method='prox')

In [26]:
utc_amici.floating_species_list, amici_results.get('floating_species').keys()

In [27]:
[spec.name for spec in utc_copasi.sbml_model.getListOfSpecies()]

In [28]:
utc_copasi.floating_species_list

In [29]:
utc_tellurium.floating_species_list

In [30]:
utc_tellurium.simulator.plot()

In [31]:
utc_copasi._tc.plot()

In [32]:
from notebook_api.utc import generate_utc_step_outputs

In [33]:
results = generate_utc_step_outputs(omex_fp)

from process_bigraph import pp


pp(results)

In [34]:
results['results']['amici']['floating_species'].keys()

In [40]:
results['instances'][0].plot_results()

In [35]:
results['instances'][1].plot_results()

We can read the data from any of the saved files:

In [36]:
csv_result_dir = list(map(lambda d: os.path.join(save_dir, d), os.listdir(save_dir)))

csv_result_dir

In [37]:
import pandas as pd 

dfs = list(map(pd.read_csv, csv_result_dir))

In [38]:
lac_ci_df = pd.read_csv(csv_result_dir[0])

In [39]:
lac_ci_df

In [5]:
from process_bigraph import Composite


doc = {
    'amici': {
        '_type': 'step',
        'address': 'local:utc-amici',
        'config': {
            'model': {
                'model_source': omex_fp
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species': ['amici_floating_species_store'],
            'model_parameters': ['amici_model_parameters_store'],
            'reactions': ['amici_reactions_store']
        },
        'outputs': {
            'time': ['time_store'],
            'floating_species': ['amici_floating_species_store']
        }
    },
    'copasi': {
        '_type': 'step',
        'address': 'local:utc-copasi',
        'config': {
            'model': {
                'model_source': omex_fp
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species': ['copasi_floating_species_store'],
            'model_parameters': ['copasi_model_parameters_store'],
            'reactions': ['copasi_reactions_store']
        },
        'outputs': {
            'time': ['time_store'],
            'floating_species': ['copasi_floating_species_store']
        }
    },
    'tellurium': {
        '_type': 'step',
        'address': 'local:utc-tellurium',
        'config': {
            'model': {
                'model_source': omex_fp
            }
        },
        'inputs': {
            'time': ['time_store'],
            'floating_species': ['tellurium_floating_species_store'],
            'model_parameters': ['tellurium_model_parameters_store'],
            'reactions': ['tellurium_reactions_store']
        },
        'outputs': {
            'time': ['time_store'],
            'floating_species': ['tellurium_floating_species_store']
        }
    },
    'comparison': {
        '_type': 'step',
        'address': 'local:utc-comparator',
        'config': {
            'simulators': ['amici', 'copasi', 'tellurium'],
        },
        'inputs': {
            'time': ['time_store'],
            'amici_floating_species': ['amici_floating_species_store'],
            'copasi_floating_species': ['copasi_floating_species_store'],
            'tellurium_floating_species': ['tellurium_floating_species_store'],
        },
        'outputs': {
            'mean_squared_error': ['mean_squared_error_store'],
            'proximity': ['proximity_store'],
            'output_data': ['output_data_store']
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'mean_squared_error': 'tree[string]',
                'proximity': 'tree[string]',
                'output_data': 'tree[string]'
            }
        },
        'inputs': {
            'mean_squared_error': ['mean_squared_error_store'],
            'proximity': ['proximity_store'],
            'output_data': ['output_data_store']
        }
    }
}

In [7]:
result = wf.run(1)

In [8]:
result = wf.gather_results()

In [9]:
result